In [26]:
import pandas as pd
import gzip
import json
import re
import datetime
import numpy as np

In [27]:
def get_data_from_date(specific_date):
    return all_data[(all_data["Date"] == end_date)]

In [219]:
class Coinmarketcap(object):
    @staticmethod
    def load_data(source_datapath):
        all_data = pd.read_csv(source_datapath, sep='\t', header=0, encoding="utf-8", dtype={'Market Cap': np.float64, 'Total Market Cap': np.float64})
        all_data["Market Cap Share"] = all_data["Market Cap"]/all_data["Total Market Cap"]
        return all_data

    def __init__(self, source_datapath):
        self.data = self.load_data(source_datapath)
    
    def get_coin_price(self, date, coin_symbol):
        try:
            return float(self.data[(self.data["Date"] == date) & (self.data["Symbol"] == coin_symbol)]["Price"].values[0])
        except:
            print("ERROR: price not found for coin %s (%s)" % (symbol, date))
            return 0

    def coins_to_usd(self, date, coins):
        amount_usd = 0
        for coin_symbol, amount in coins.items():
            coin_usd = self.get_coin_price(date, symbol)
            amount_usd += amount * coin_usd
        return amount_usd
    
    def buy_coin(self, date, symbol, amount_usd):
        return amount_usd/self.get_coin_price(date, symbol)
    
    def buy_coins(self, date, symbols, amount_usd, strategy="even"):
        symbols_length = len(symbols)
        coins = {}
        if strategy == "market_cap":
            coins_market_cap = {}
            total_market_cap= 0
            for coin_symbol in symbols:
                coins_market_cap[coin_symbol] = self.get_coin_market_cap(date, coin_symbol)
                total_market_cap += coins_market_cap[coin_symbol]
        for coin_symbol in symbols:
            coin_amount_usd = 0
            if strategy == "even":
                coin_amount_usd = (amount_usd/symbols_length)
            elif strategy == "market_cap":
                coin_amount_usd = (amount_usd*(coins_market_cap[coin_symbol]/total_market_cap))
            coins[coin_symbol] = self.buy_coin(date, coin_symbol, coin_amount_usd)
        return coins
    
    def sell_coins(self, date, coins):
        amount_usd = 0
        for coin_symbol, coin_amount in coins.items():
            coin_price = self.get_coin_price(date, coin_symbol)
            amount_usd += coin_price*coin_amount
        return amount_usd
    
    def get_coin_market_cap(self, date, coin_symbol):
        return float(self.data[(self.data["Date"] == date) & (self.data["Symbol"] == coin_symbol)]["Market Cap"].values[0])
            
    def get_positions_range(self, date, first_position, last_position):
        coins_data = self.data[(self.data["Date"] == end_date) & (date_data["Pos"] <= last_position) & (date_data["Pos"] >= first_position)]
        for index, _ in coins_data["Symbol"].iteritems():
            yield coins_data["Symbol"][index]
    
    def dates(self):
        return sorted(self.data.Date.unique())
    
    def print_returns(self, initial_amount, initial_date, final_amount, final_date):
        print("%s: %s USD" % (initial_date, round(initial_amount, 2)))
        print("%s: %s USD" % (end_date, round(final_amount, 2)))
        print("----------")
        print("%sX" % round(final_amount/initial_amount, 1))
        
    
        
        

# Initialize

In [220]:
market = Coinmarketcap("data/coinmarketcap_formatted.tsv")
all_dates = market.dates()
weeks_ago = 54
initial_amount = 1000
start_date = all_dates[-weeks_ago]
end_date = all_dates[-1]

# Strategies

In [221]:
print("With an initial of %s USD from %s to %s" % (initial_amount, start_date, end_date))

With an initial of 1000 USD from 2016-04-10 to 2017-04-23


## Buy first 5 coins evenly and forget

In [222]:
symbols_to_buy = market.get_positions_range(start_date, 1, 5)
coins = market.buy_coins(start_date, list(symbols_to_buy), initial_amount, strategy="even")
final_amount = market.sell_coins(end_date, coins)
market.print_returns(initial_amount, start_date, final_amount, end_date)

even
even
even
even
even
2016-04-10: 1000 USD
2017-04-23: 5703.29 USD
----------
5.7X


## Buy 50% Bitcoin 50% Top(2-5)

In [223]:
symbols_to_buy = list(market.get_positions_range(start_date, 1, 1))
coins = market.buy_coins(start_date, symbols_to_buy, initial_amount/2, strategy="even")
symbols_to_buy = list(market.get_positions_range(start_date, 2, 5))
top_coins = market.buy_coins(start_date, symbols_to_buy, initial_amount/2, strategy="even")
coins = {**coins, **top_coins}
final_amount = market.sell_coins(end_date, coins)
market.print_returns(initial_amount, start_date, final_amount, end_date)

even
even
even
even
even
2016-04-10: 1000 USD
2017-04-23: 4657.87 USD
----------
4.7X


## Buy Top 5 by market cap

In [224]:
symbols_to_buy = market.get_positions_range(start_date, 1, 5)
coins = market.buy_coins(start_date, list(symbols_to_buy), initial_amount, strategy="market_cap")
print(coins)
final_amount = market.sell_coins(end_date, coins)
market.print_returns(initial_amount, start_date, final_amount, end_date)

market_cap
market_cap
market_cap
market_cap
market_cap
{'BTC': 2.0334483991560117, 'ETH': 10.40862907924173, 'XRP': 4543.445351096338, 'LTC': 5.980036490714312, 'DASH': 0.8394024048606977}
2016-04-10: 1000 USD
2017-04-23: 3280.62 USD
----------
3.3X


In [4]:
def convert_coins_to_usd(date_data, coins):
    amount_usd = 0
    for coin_symbol, number_of_coins in coins.items():
        try:
            coin_usd = float(date_data[date_data["Symbol"] == coin_symbol]["Price"].values[0])
        except:
            coin_usd = 0
        amount_usd += number_of_coins * coin_usd
    return amount_usd

In [5]:
def buy_coins(date_data, amount_usd, buy_x_first, skip_x_first=0):
    max_pos = buy_x_first + skip_x_first
    coins_to_buy_data = date_data[(date_data["Pos"] <= max_pos) & (date_data["Pos"] > skip_x_first)]
    coins = {}
    coins_usd = {}
    for index, _ in coins_to_buy_data["Price"].iteritems():
        coin_usd = float(date_data["Price"][index])
        coin_symbol = date_data["Symbol"][index]
        normalize_market_cap_share = coins_to_buy_data["Market Cap Share"][index]/coins_to_buy_data["Market Cap Share"].sum().astype(np.float64)
        total_of_coin = 1.0 * normalize_market_cap_share * amount_usd / coin_usd
        coins[coin_symbol] = total_of_coin
        coins_usd[coin_symbol] = total_of_coin * coin_usd
    return coins, coins_usd

def buy_coins_equaly(date_data, amount_usd, buy_x_first, skip_x_first=0):
    max_pos = buy_x_first + skip_x_first
    coins_to_buy_data = date_data[(date_data["Pos"] <= max_pos) & (date_data["Pos"] > skip_x_first)]
    coins = {}
    coins_usd = {}
    for index, _ in coins_to_buy_data["Price"].iteritems():
        coin_usd = float(date_data["Price"][index])
        coin_symbol = date_data["Symbol"][index]
        total_of_coin = 1.0 * (amount_usd/buy_x_first) / coin_usd
        coins[coin_symbol] = total_of_coin
        coins_usd[coin_symbol] = total_of_coin * coin_usd
    return coins, coins_usd

def buy_coins_factor(date_data, amount_usd, buy_x_first, skip_x_first=0, factor=3):
    mult = factor
    step = (mult-1)/buy_x_first
    max_pos = buy_x_first + skip_x_first
    coins_to_buy_data = date_data[(date_data["Pos"] <= max_pos) & (date_data["Pos"] > skip_x_first)]
    coins = {}
    coins_usd = {}
    new_total_market_cap = 0
    curr_mult = 1
    for index, _ in coins_to_buy_data["Price"].iteritems():
        new_total_market_cap += curr_mult*coins_to_buy_data["Market Cap"][index]
        curr_mult += step

    curr_mult = 1
    for index, _ in coins_to_buy_data["Price"].iteritems():
        coin_usd = float(date_data["Price"][index])
        coin_symbol = date_data["Symbol"][index]
        normalize_market_cap_share = curr_mult*coins_to_buy_data["Market Cap"][index]/new_total_market_cap
        total_of_coin = 1.0 * normalize_market_cap_share * amount_usd / coin_usd
        coins[coin_symbol] = total_of_coin
        coins_usd[coin_symbol] = total_of_coin * coin_usd
        curr_mult += step
    return coins, coins_usd
    

In [6]:
def calculate_returns(buy_x_first, start_date, end_date, coins, skip_x_first=0):
    max_pos = buy_x_first + skip_x_first
    to_return = []
    last_coins = {}
    dates = sorted(list(all_data[(all_data["Date"] >= start_date) & (all_data["Date"] <= end_date)]["Date"].unique()))
    for date_ in dates:
        ## Convert to usd
        date_data = all_data[(all_data["Date"] == date_)]
        amount_usd = convert_coins_to_usd(date_data, coins)
        to_return.append([date_, round(amount_usd, 2)])

        # Buy top X coins
        coins_to_buy_data = date_data[(date_data["Pos"] <= max_pos) & (date_data["Pos"] > skip_x_first)]
        coins, coins_usd = buy_coins(date_data, amount_usd, buy_x_first, skip_x_first=skip_x_first)
        last_coins = [coins, coins_usd]
        
    return to_return, last_coins


In [7]:
#You have to select a date that is available in the dataset
def print_returns_usings_topx(start_date, end_date, initial_coins, buy_x_firsts, show_coins_to_buy=False, skip_x_first=0):
    print("From: %s to %s" %(start_date, end_date))
    initial = True
    coins_to_buy = {}
    print("Top X Coins\tFinal Growth")
    for buy_x_first in buy_x_firsts:
        calculated_values, coins_to_buy = calculate_returns(buy_x_first, start_date, end_date, initial_coins, skip_x_first=skip_x_first)
        print("%s\t%s" % (buy_x_first, "%s%%" % round(100* (calculated_values[-1][1]/calculated_values[0][1]-1),2)))
    if show_coins_to_buy:
        print("")
        print("\t".join(["Coin", "Total Coins", "Coin Price USD", "Total USD"]))
        for coin in coins_to_buy[0].keys():
            print(
                "\t".join([
                    str(coin),
                    str(coins_to_buy[0][coin]),
                    str(coins_to_buy[1][coin]/coins_to_buy[0][coin]), 
                    str(coins_to_buy[1][coin])
                ])
            )


In [8]:
start_date, end_date = "2017-01-01", "2017-04-16"
initial_coins = {
    "BTC": 10
}
buy_x_firsts = [1,2,3,5,10,20,100]
skip_x_first = 2
print_returns_usings_topx(start_date, end_date, initial_coins, buy_x_firsts, skip_x_first=skip_x_first)


From: 2017-01-01 to 2017-04-16
Top X Coins	Final Growth
1	112.27%
2	170.38%
3	232.66%
5	231.19%
10	215.59%
20	200.29%
100	199.04%


In [9]:
start_date, end_date = "2017-01-01", "2017-04-16"
initial_coins = {
    "BTC": 10
}
buy_x_firsts = [1,2,3,5,10,20,100]
skip_x_first = 0
print_returns_usings_topx(start_date, end_date, initial_coins, buy_x_firsts)


From: 2017-01-01 to 2017-04-16
Top X Coins	Final Growth
1	22.43%
2	43.59%
3	45.79%
5	51.35%
10	54.3%
20	54.91%
100	57.5%


In [10]:
start_date, end_date = "2017-04-23", "2017-04-23"
initial_coins = {
    "BTC": 45
}
buy_x_firsts = [5]
print_returns_usings_topx(start_date, end_date, initial_coins, buy_x_firsts, show_coins_to_buy=True)



From: 2017-04-23 to 2017-04-23
Top X Coins	Final Growth
5	0.0%

Coin	Total Coins	Coin Price USD	Total USD
BTC	33.5320383522	1224.68	41066.0167292
ETH	187.265866776	48.47	9076.77656265
XRP	77988.1300197	0.031566	2461.7733122
LTC	104.523714191	13.85	1447.65344155
DASH	14.9193678375	70.94	1058.37995439


In [17]:
start_date = "2017-01-01"
amount_usd = 10000
buy_x_first = 5
skip_x_first = 0
date_data = all_data[(all_data["Date"] == start_date)]
coins, coins_usd = buy_coins(date_data, amount_usd, buy_x_first, skip_x_first)
coins_eq, coins_eq_usd = buy_coins_equaly(date_data, amount_usd, buy_x_first, skip_x_first)
coins_factor, coins_factor_usd = buy_coins_factor(date_data, amount_usd, buy_x_first, skip_x_first, factor=2)

end_date = "2017-04-23"
date_data = all_data[(all_data["Date"] == end_date)]
print(convert_coins_to_usd(date_data, coins))
#print(convert_coins_to_usd(date_data, coins_eq))
#print(convert_coins_to_usd(date_data, coins_factor))

15454.9675419
